# Masakhane - Traduction automatique inversée pour les langues africaines (Via JoeyNMT)

> ## NB
>### - Le but de ce notebook est de construire des modèles qui traduisent les langues africaines (source_language) *en* anglais (target_language). Cela nous permettra à l'avenir de pouvoir faire des traductions d'une langue africaine à l'autre. Si vous souhaitez traduire *de* l'anglais, veuillez utiliser [ce](https://github.com/masakhane-io/masakhane-traduction-automatique/blob/main/notebook_debutant_formation_a_partir_de_lAnglais.ipynb) notebook débutant à la place.


>### - Nous appelons cela entrainement inversé car, normalement, nous construisons des modèles qui font des traductions de la langue source (anglais) vers la langue cible. Mais dans ce cas, nous faisons l'inverse : nous construisons des modèles qui font des traductions de la langue cible vers la langue source (anglais).

## Note avant de debuter:
### - L'idée est que vous soyez capable de faire des changements legers à ce code pour avoir un resultat pour votre corpus de traduction. 

### - tl;dr: Les commentaires **"TODO"** vous donnent des directives sur ce que vous devez modifier

### - Si vous voulez avoir une idée de ce que vous faites, lisez le texte et jetez un coup d'œil aux liens.

### - Avec 100 époques, l'exécution dans Google Colab devrait prendre environ 7 heures.

### - Une fois que vous avez obtenu un résultat pour votre langue, veuillez joindre et envoyer par courriel votre notebook qui l'a généré à masakhanetranslation@gmail.com.

### - Si vous y tenez et que vous en avez l'occasion, il serait formidable de faire un bref historique de votre langue. Voir les exemples dans [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Récupérer vos données et créer un corpus parallèle

Si vous voulez utiliser les données JW300 référencées sur le site de Masakhane ou dans notre repo GitHub, vous pouvez utiliser `opus-tools` pour convertir les données dans un format pratique. `opus_read` de ce paquetage fournit un outil pratique pour lire les fichiers XML natifs alignés et les convertir au format TMX. Cet outil peut également être utilisé pour récupérer les fichiers pertinents d'OPUS à la volée et pour filtrer les données si nécessaire. [Lisez la documentation](https://pypi.org/project/opustools-pkg/) pour plus de détails.

Une fois que vous avez vos fichiers de corpus au format TMX (une structure xml qui comprendra les phrases dans votre langue cible et votre langue source dans un seul fichier), nous vous recommandons de les lire dans un dataframe pandas. Heureusement, Jade a écrit un paquet bidon `tmx2dataframe` qui convertit votre fichier tmx en un dataframe pandas. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# TODO: Définissez vos langues source et cible. Gardez à l'esprit que ces langues utilisent traditionnellement les codes de langue que l'on trouve ici :
# Ceux-ci deviendront également les suffixes de tous les fichiers de vocabulaire et de corpus utilisés tout au long du projet.
import os
source_language = "en"
target_language = "sw" 
lc = False  # Si Vrai, transformer les données en minuscules.
seed = 42  # Graine aléatoire pour le mélange des données.
tag = "baseline" # Donnez un nom unique à votre dossier - ceci afin de vous assurer que vous n'écrasez pas les modèles que vous avez déjà soumis.

os.environ["src"] = source_language # Les défini en bash également, puisque nous utilisons souvent des scripts bash
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# Ceci permet de l'enregistrer dans un dossier de notre gdrive plutot!
!mkdir -p "/content/drive/My Drive/masakhane/$tgt-$src-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (target_language, source_language, tag)

In [4]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-sw-baseline


In [5]:
# Installer opus-tools
! pip install opustools-pkg

In [ ]:
# Telechargement de notre corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extraction du fichier du corpus
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-sw.xml.gz not found. The following files are available for downloading:

   8 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-sw.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  94 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/sw.zip

 365 MB Total size
./JW300_latest_xml_en-sw.xml.gz ... 100% of 8 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_sw.zip ... 100% of 94 MB


In [ ]:
# Telecharger le jeu de données global de test.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# Et le jeu de test specifique pour cette paire de langues.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-10-28 06:26:47--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.1’


test.en-any.en.1      0%[                    ]       0  --.-KB/s               
test.en-any.en.1    100%[===================>] 271.28K  --.-KB/s    in 0.03s   

2020-10-28 06:26:47 (9.82 MB/s) - ‘test.en-any.en.1’ saved [277791/277791]

--2020-10-28 06:26:47--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-sw.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)

In [9]:
# Lire les données de test pour les filtrer des données d'entrainement et de développement.
# Stocker la partie anglaise dans un set pour des verifications rapides du filtrage.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Chargement de {} phrases de test à filtrer des données d\'entrainement\nReussi !'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [10]:
import pandas as pd

# fichier TMX vers dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collectez les numéros de ligne de la partie source pour sauter les mêmes lignes pour la partie cible.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Sauter les phrases contenues dans le jeu de test
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Ajouter au corpus ssi la source correspondante n'a pas été sautée
        if j not in skip_lines:
            target.append(line.strip())
    
print('Chargement des données et ignorance des lignes {}/{} puisqu\'elles sont contenues dans l\'ensemble de test.\nReussi!'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# si vous obtenez TypeError : data argument can't be an iterator c'est à cause de votre version de zip. exécutez la ligne ci-dessous
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 6478/979526 lines since contained in test set.


,source_sentence,target_sentence
0,“ Look !,“ Tazama !
1,I Am Making All Things New ”,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
2,The above is a promise from God that has been ...,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo ...


## Pre-traitement et exportation

C'est généralement une bonne idée de supprimer les traductions en double et les traductions contradictoires du corpus. En pratique, ces corpus publics en comportent un certain nombre qui doivent être nettoyés.

De plus, nous allons diviser nos données dans dev/test/train et les exporter vers le système de fichiers.

In [11]:
# supprimer les traductions en double
df_pp = df.drop_duplicates()

# supprimer les traductions contradictoires
# (ceci optionnel et vous pourriez vouloir le commenter 
# selon la taille de votre corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Mélangez les données pour éliminer les biais dans la sélection du jeu de developpement.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Installez fuzzy wuzzy pour supprimer les phrases "presque en double" dans les ensembles de test et d'apprentissage.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# réinitialiser l'index de l'ensemble d'apprentissage après le filtrage précédent
df_pp.reset_index(drop=False, inplace=True)

# Supprimez les échantillons de l'ensemble d'apprentissage s'ils se 'chevauchent presque' avec les échantillons de l'ensemble de test.

# Fonction de filtrage. Ajustez le tampon (pad) pour réduire les correspondances candidates
# dans une certaine longueur de caractères de l'échantillon donné.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144796 sha256=a672b4c48e25c303e96476a57daff44ea0e5e6d99efc41a4a1c494ca2eefcd39
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [23]:
# start_time = time.time()
# ### itérer sur les lignes d'un dataframe pandas n'est pas recommandé, utilisons le traitement multiple pour appliquer la fonction.

# with Pool(cpu_count()-1) as pool:
#     scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
# hours, rem = divmod(time.time() - start_time, 3600)
# minutes, seconds = divmod(rem, 60)
# print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# # Filtrer les échantillons qui se "chevauchent presque".
# df_pp = df_pp.assign(scores=scores)
# df_pp = df_pp[df_pp['scores'] < 95]

In [12]:
# Cette section effectue la séparation entre train/dev pour les corpus parallèles et les enregistre dans des fichiers séparés.
# Nous utilisons 1000 dev test et l'ensemble de test donné.
import csv

# Faire le split entre dev/train et créer des corpus parallèles
num_dev_patterns = 1000

# Optionnel : mettre les corpus en minuscules - cela facilitera la généralisation, mais sans la casse correcte.
if lc:  # Julia: rendre la minuscularisation facultative
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: les ensembles de test sont déjà générés
dev = df_pp.tail(num_dev_patterns) # Herman: Erreur dans l'original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Vérifiez le format ci-dessous. Il ne doit pas y avoir de guillemets supplémentaires ou de caractères bizarres.
! head train.*
! head dev.*

==> train.en <==
Besides , being single gives one the opportunity to do so many great things !
A Logical Conclusion
I still feel the pain of the loss , but thanking Jehovah for having had a good marriage and for the privilege of serving him with someone who deeply loved him has improved my outlook . ”
If we do his will , he blesses and supports us in this life and offers us the certain hope of an even better life to come .
We are determined to continue providing a rich supply of enlightening and appealing information ​ — both in print and online — ​ to benefit our many readers who respect the Bible and who want to know what it really teaches .
Well , in the past decade , the number of active Witnesses increased from slightly fewer than 3,800,000 to almost 6,000,000 .
Actually , according to the Bible , there is no spiritual part of a person that survives the death of the body .
The enemy may aim a blow by attacking Bible truths that are fundamental to your faith .
When Saul returned to



---


## Installation de JoeyNMT

JoeyNMT est un paquet de Traduction Automatique Neuronale (NMT) simple et minimaliste, utile pour l'apprentissage et l'enseignement. Consultez la documentation de JoeyNMT [ici](https://joeynmt.readthedocs.io)  

In [13]:
# Installer JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Installer Pytorch avec support GPU v1.7.1.
! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'joeynmt'...
remote: Enumerating objects: 2546, done.
remote: Total 2546 (delta 0), reused 0 (delta 0), pack-reused 2546
Receiving objects: 100% (2546/2546), 2.74 MiB | 30.50 MiB/s, done.
Resolving deltas: 100% (1731/1731), done.
Processing /content/joeynmt
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 276kB 15.2MB/s 
     |████████████████████████████████| 327kB 28.8MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 215kB 23.4MB/s 
     |████████████████████████████████| 747kB 29.0MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.3-cp36-none-any.whl size=77279 sha256=250f6b09f2bf7e40185d94579f80cf1963bde82939acd6255f57d60064c5bccb
  Stored in directory: /tmp/pip-ephem-wheel-cache-6u2rlugr/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-l

# Pré-traitement des données en tokens BPE

- L'une des améliorations les plus puissantes pour les langues agglutinantes (une caractéristique de la plupart des langues bantoues) est l'utilisation de la tokenisation BPE [(Sennrich, 2015)](https://arxiv.org/abs/1508.07909).

- Il a également été démontré qu'en optimisant le nombre de codes BPE, nous améliorons considérablement les résultats pour les langues à faibles ressources [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Ci-dessous nous avons les scripts pour faire la tokenisation BPE de nos données. Nous utilisons 4000 tokens comme recommandé par [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). Vous n'avez pas besoin de modifier quoi que ce soit. Il suffit d'exécuter ce qui suit. 

In [14]:
# L'utilisation d'une méthode différente de tokénisation a permis d'améliorer considérablement les performances de NMT. 
# Habituellement, NMT procède à la tokenisation par mots. Cependant, l'utilisation d'une méthode appelée BPE a permis d'améliorer considérablement les performances.

# Faire un NMT de sous-mots
from os import path
os.environ["src"] = source_language # Les définir en bash également, puisque nous utilisons souvent des scripts bash
os.environ["tgt"] = target_language

# Apprendre les BPE sur les données d'entrainement.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Appliquer les splits BPE aux données de développement et de test.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Créer un répertoire, déplacer tout ce à quoi nous tenons vers le bon endroit.
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Déplacez également tout ce qui nous intéresse vers un emplacement monté dans google drive (pertinent si vous travaillez sur colab) à gdrive_path.
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Créer le vocabulaire avec build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Echantillon de resultat
! echo "BPE Swahili Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.sw     test.en-any.en.1  train.bpe.sw
dev.bpe.en	dev.sw	     test.en	     test.sw	       train.en
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.en      train.sw
bpe.codes.4000	dev.en	     test.bpe.sw     test.en-any.en.1  train.bpe.sw
dev.bpe.en	dev.sw	     test.en	     test.sw	       train.en
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.en      train.sw
BPE Swahili Sentences
N@@ g@@ ao kubwa ya imani ( T@@ azama f@@ ungu la 12 hadi 14 )
K@@ of@@ ia ya chu@@ ma ya wo@@ ko@@ vu ( T@@ azama f@@ ungu la 15 hadi 18 )
N@@ im@@ et@@ ambua kwamba watu hu@@ it@@ ikia vizuri wanapo@@ ona uki@@ zungum@@ zia habari za Biblia kwa sha@@ uku na unapo@@ fanya yote una@@ yo@@ weza kuwasaidia . ”
Up@@ anga wa roho ( T@@ azama f@@ ungu la 19 na 20 )
Kwa msaada wa Yehova tunaweza kus@@ im@@ ama im@@ ara na kum@@ p@@ inga !
Combined BPE Vocab
с@@
ḥ
т@@
и
х@@
м
і@@
д@@
і
➊


In [15]:
# Déplacez également tout ce qui nous intéresse vers un emplacement monté dans google drive (pertinent si vous travaillez sur colab) à gdrive_path.
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.sw     test.en-any.en.1  train.bpe.sw
dev.bpe.en	dev.sw	     test.en	     test.sw	       train.en
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.en      train.sw


# Creation de la config de JoeyNMT

JoeyNMT nécessite une configuration yaml. Nous fournissons un modèle ci-dessous. Nous avons également défini un certain nombre de valeurs par défaut, avec lesquelles vous pouvez jouer !

- Nous avons utilisé l'architecture Transformer 
- Nous avons fixé notre dropout à un niveau raisonnablement élevé : 0,3 (recommandé dans [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Des choses avec lesquelles il vaut la peine de s'amuser :
- La taille du lot (batch size) (il est également recommandé de la modifier pour les langues à faibles ressources).
- Le nombre d'époques (epochs) (nous l'avons fixé à 30 pour qu'il fonctionne en une heure environ, à des fins de test).
- Les options du décodeur (beam_size, alpha)
- Les métriques d'évaluation (BLEU versus Crhf4)

In [24]:
# Ceci crée le fichier de configuration pour notre système JoeyNMT. Comme cela peut sembler compliqué, nous avons fourni quelques paramètres utiles à mettre à jour.
# (Vous pouvez bien sûr jouer avec tous les paramètres si vous le souhaitez !)

name = '%s%s' % (target_language, source_language)
# gdrive_path = os.environ["gdrive_path"]

# Créer la config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: essayez de passer de plateau au scheduling de Noam
    patience: 5                     # Pour plateau : diminuer le taux d'apprentissage (learning rate) par le facteur de diminution (decrease_factor) si le score de validation ne s'est pas amélioré pendant ce nombre de tours de validation.
    learning_rate_factor: 0.5       # facteur pour le scheduler Noam (utilisé avec Transformer)
    learning_rate_warmup: 1000      # étapes d'échauffement pour le scheduler Noam (utilisé avec Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 5                  # TODO: Diminuer pour faire des essais et vérifier le fonctionnement. Une trentaine de minutes suffisent pour vérifier si le système fonctionne.
    validation_freq: 1000          # TODO: Définir à au moins une fois par époque.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer"
    overwrite: True              # TODO: Mettez la valeur True si vous voulez écraser les modèles éventuellement existants. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Augmentez à 8 pour les données plus volumineuses.
        embeddings:
            embedding_dim: 256   # TODO: Augmentez à 512 pour les données plus volumineuses.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Augmentez à 512 pour les données plus volumineuses.
        ff_size: 1024            # TODO: Augmentez à 2048 pour les données plus volumineuses.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Augmentez à 8 pour les données plus volumineuses.
        embeddings:
            embedding_dim: 256    # TODO: Augmentez à 512 pour les données plus volumineuses.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Augmentez à 512 pour les données plus volumineuses.
        ff_size: 1024            # TODO: Augmentez à 2048 pour les données plus volumineuses.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Entrainer le Modele

Cette simple ligne de joeynmt exécute l'entraînement en utilisant la configuration que nous avons faite ci-dessus

In [ ]:
# Entrainer le modèle
# Vous pouvez appuyer sur Ctrl-C pour arrêter. Et ensuite, exécutez la cellule suivante pour sauvegarder vos points de contrôle ! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2020-10-28 08:03:31,578 Hello! This is Joey-NMT.
2020-10-28 08:03:31.709513: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-28 08:03:32,785 Total params: 12219392
2020-10-28 08:03:32,786 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_

In [ ]:
# Copiez les modèles créés depuis le stockage du notebook vers google drive pour un stockage permanent. 
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "$gdrive_path/models/${src}${tgt}_reverse_transformer/"

In [ ]:
# Sortie de notre score de validation
! cat "$gdrive_path/models/${tgt}${src}_reverse_transformer/validations.txt"

In [ ]:
# Tester notre modèle
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${tgt}${src}_reverse_transformer/config.yaml"